<a href="https://colab.research.google.com/github/sfwtopoulos/stocks/blob/master/Stocks_seq_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
pip install -q talos

     |████████████████████████████████| 61kB 2.8MB/s 
     |████████████████████████████████| 109.2MB 78kB/s 
     |████████████████████████████████| 8.7MB 30.5MB/s 
     |████████████████████████████████| 378kB 43.3MB/s 
     |████████████████████████████████| 491kB 42.3MB/s 
     |████████████████████████████████| 3.2MB 38.2MB/s 
     |████████████████████████████████| 839kB 42.7MB/s 
ERROR: wrangle 0.6.7 has requirement scipy==1.2, but you'll have scipy 1.4.1 which is incompatible.


In [4]:
pip install -q wget

In [5]:
import pandas as pd
import matplotlib.pyplot as plt
import pandas as pd
import math
import warnings
import numpy as np
import time
import sys

import pandas_datareader as pdr
#%tensorflow_version 2.x
# Restart runtime using 'Runtime' -> 'Restart runtime...'
%tensorflow_version 1.x
import tensorflow as tf
import keras


# from keras.layers import LSTM
# from keras.models import Sequential
# from keras.layers.wrappers import TimeDistributed
# from keras.layers.wrappers import TimeDistributed
# from keras.layers.core import Dense, Activation, Dropout
# from keras.preprocessing.sequence import pad_sequences

from tensorflow.python.keras.layers import Dense, Activation, Dropout, Input
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import LSTM
from tensorflow.python.keras.layers.wrappers import TimeDistributed
from tensorflow.python.keras.layers.wrappers import TimeDistributed
from tensorflow.python.keras.preprocessing.sequence import pad_sequences


from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import seaborn as sns
import matplotlib.pyplot as plt

import os
import wget
from zipfile import ZipFile

warnings.filterwarnings('ignore',category=FutureWarning)
warnings.filterwarnings('ignore', category=DeprecationWarning)

/usr/local/lib/python3.6/dist-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


TensorFlow 1.x selected.


Using TensorFlow backend.


In [0]:
# from keras.optimizers import Adam, Nadam
# from keras.activations import softmax
# from keras.losses import categorical_crossentropy, logcosh
# from talos.utils import lr_normalizer
import talos
from tensorflow.python.keras.optimizers import Adam, Nadam
from tensorflow.python.keras.activations import softmax
from tensorflow.python.keras.losses import categorical_crossentropy, logcosh
from talos.utils import lr_normalizer

In [7]:
print(keras.__version__)
print(tf.__version__)
#print(tensorflow.compat.v2.__version__)

2.3.0
1.15.2


In [8]:

large=True;

if not large:
  #Small Dataset
  #Data import from git repo
  print("Small Dataset Loading")
  url = 'https://raw.githubusercontent.com/sfwtopoulos/stocks/master/stocks_dataset/combined.csv'
  #df1 = pd.read_csv(url, error_bad_lines=False)
  dfstocks = pd.read_csv(url, sep=',')
  dfstocks=dfstocks.rename({' AAPL':'Company', ' Close/Last':'Close', ' Volume':'Volume', ' Open':'Open', ' High':'High', ' Low':'Low'}, axis=1);
else:
  #Large Dataset
  print("Large Dataset Loading")
  url = 'https://raw.githubusercontent.com/sfwtopoulos/stocks/master/stocks_dataset/combined_large.zip'
  wget.download(url, '.')
  # Create a ZipFile Object and load sample.zip in it
  with ZipFile('combined_large.zip', 'r') as zipObj:
    # Extract all the contents of zip file in current directory
    zipObj.extractall()
  print(os.listdir("."))
  dfstocks = pd.read_csv('combined_large.csv', sep=',') 
  dfstocks=dfstocks[~dfstocks.Date.str.contains("nan")]
  dfstocks=dfstocks[~dfstocks.Date.str.contains("Low")]
  dfstocks=dfstocks[~dfstocks.Date.str.contains("N/A")]
  dfstocks=dfstocks[~dfstocks.Date.str.contains("date")]
  dfstocks.Date=pd.to_datetime(dfstocks.Date)
  dfstocks['Date'] = dfstocks['Date'].dt.strftime('%Y-%m-%d')
  dfstocks=dfstocks.rename({' AA':'Company', ' Close/Last':'Close', ' Volume':'Volume', ' Open':'Open', ' High':'High', ' Low':'Low'}, axis=1);

Large Dataset Loading
['.config', 'combined_large.zip', 'combined_large.csv', 'sample_data']


In [0]:
# #Large Dataset
# url = 'https://raw.githubusercontent.com/sfwtopoulos/stocks/master/stocks_dataset/combined_large.zip'
# wget.download(url, '.')
# # Create a ZipFile Object and load sample.zip in it
# with ZipFile('combined_large.zip', 'r') as zipObj:
#   # Extract all the contents of zip file in current directory
#   zipObj.extractall()
# print(os.listdir("."))
# dfstocks = pd.read_csv('combined_large.csv', sep=',') 
# dfstocks=dfstocks[~dfstocks.Date.str.contains("nan")]
# dfstocks=dfstocks[~dfstocks.Date.str.contains("Low")]
# dfstocks=dfstocks[~dfstocks.Date.str.contains("N/A")]
# dfstocks=dfstocks[~dfstocks.Date.str.contains("date")]
# dfstocks.Date=pd.to_datetime(dfstocks.Date)
# dfstocks['Date'] = dfstocks['Date'].dt.strftime('%Y-%m-%d')
# dfstocks=dfstocks.rename({' AA':'Company', ' Close/Last':'Close', ' Volume':'Volume', ' Open':'Open', ' High':'High', ' Low':'Low'}, axis=1);
# #dfstocks=dfstocks.head(10)

In [10]:
type(dfstocks['Date'])
dfstocks.dtypes

Date       object
Close      object
Volume     object
Open       object
High       object
Low        object
Company    object
dtype: object

In [11]:
for col in dfstocks.columns: 
    print(col) 

Date
Close
Volume
Open
High
Low
Company


In [12]:
dfstocks

,Date,Close,Volume,Open,High,Low,Company
0,2018-02-11,36.93000031,2681700,37,37.68999863,36.40999985,AA
1,2018-01-11,36.77999878,2756200,35.43999863,36.88000107,35.24000168,AA
2,2018-10-31,34.99000168,3380700,34.81999969,35.82099915,34.59999847,AA
3,2018-10-30,34.15000153,2193400,33.65000153,34.47999954,33.4129982,AA
4,2018-10-29,33.81999969,2219600,34.59999847,35.11999893,33.40000153,AA
...,...,...,...,...,...,...,...
304988,1973-09-05,0.469444454,79200,0.469444454,0.48888889,0.469444454,USB
304989,1973-08-05,0.469444454,90000,0.469444454,0.480555564,0.469444454,USB
304990,1973-07-05,0.461111099,43200,0.469444454,0.480555564,0.461111099,USB
304991,1973-04-05,0.469444454,10800,0.469444454,0.480555564,0.469444454,USB


In [0]:
#drop $sign from values
dfstocks.Close=dfstocks['Close'].astype(str)
dfstocks.Close=dfstocks.Close.apply(lambda x: x.replace('$',''))
dfstocks.Open=dfstocks['Open'].astype(str)
dfstocks.Open=dfstocks.Open.apply(lambda x: x.replace('$',''))
dfstocks.High=dfstocks['High'].astype(str)
dfstocks.High=dfstocks.High.apply(lambda x: x.replace('$',''))
dfstocks.Low=dfstocks['Low'].astype(str)
dfstocks.Low=dfstocks.Low.apply(lambda x: x.replace('$',''))

In [14]:
#drop rows containing nan or header from the csv files
dfstocks=dfstocks[~dfstocks.Low.str.contains("nan")]
dfstocks=dfstocks[~dfstocks.Low.str.contains("Low")]
dfstocks=dfstocks[~dfstocks.Low.str.contains("N/A")]
dfstocks=dfstocks[~dfstocks.Volume.str.contains("N/A")]
dfstocks.reset_index(drop=True, inplace=True)
dfstocks.Date.count()

304968

In [15]:
#sort dataframe based on date and Company Name
dfstocks = dfstocks.sort_values(['Date', 'Company'])
dfstocks.head(100)

,Date,Close,Volume,Open,High,Low,Company
12261,1970-01-04,7.116885185,70500,7.116885185,7.148924828,7.116885185,AA
34140,1970-01-04,6.664636612,75300,6.664636612,6.71148777,6.664636612,ARNC
81366,1970-01-04,12.53962517,18500,12.50450039,12.57474995,12.50450039,CNP
93689,1970-01-04,0.775599957,2628200,0.75762248,0.778168142,0.74734962,DIS
141068,1970-01-04,0.776041687,5068800,0.76953125,0.779947937,0.76953125,GE
...,...,...,...,...,...,...,...
250575,1970-01-13,5.757882118,99200,5.757882118,5.77859354,5.716458321,MRO
12314,1970-01-14,6.808499813,57900,6.85656023,6.85656023,6.808499813,AA
34193,1970-01-14,6.371814251,61800,6.442091465,6.442091465,6.371814251,ARNC
81419,1970-01-14,10.95899963,24400,11.0994997,11.0994997,10.95899963,CNP


In [16]:
#Converting Datatypes
dfstocks.Date=pd.to_datetime(dfstocks.Date)
dfstocks.Close=pd.to_numeric(dfstocks.Close)
dfstocks.Volume=pd.to_numeric(dfstocks.Volume)
dfstocks.Open=pd.to_numeric(dfstocks.Open)
dfstocks.High=pd.to_numeric(dfstocks.High)
dfstocks.Low=pd.to_numeric(dfstocks.Low)
dfstocks.dtypes

Date       datetime64[ns]
Close             float64
Volume              int64
Open              float64
High              float64
Low               float64
Company            object
dtype: object

In [0]:
#Company Chosen
comp="ARNC"

In [18]:
#dfstocks[dfstocks['Company'].str.contains("AMZN")]
sub_split=dfstocks[dfstocks['Company'].str.contains(comp)]
sub_split.isnull().values.any()
sub_split.isna().values.any()
sub_split = sub_split.sort_values(['Date'])
sub_split.reset_index(drop=True, inplace=True)
sub_split.shape[0]

12323

In [19]:
#hidden split for validation
hidden_split=sub_split[(sub_split['Date'].dt.year==2018) | (sub_split['Date'].dt.year==2017) | (sub_split['Date'].dt.year==2016)]

#drop hidden_split from sub_split 
sub_split=sub_split[sub_split['Date'].dt.year < 2016]
hidden_split

,Date,Close,Volume,Open,High,Low,Company
11607,2016-01-02,16.214380,26539500,16.596689,16.596689,15.989493,ARNC
11608,2016-01-03,20.487242,16226500,20.509729,20.689640,20.059956,ARNC
11609,2016-01-04,21.656656,17572700,21.229370,21.679144,21.026970,ARNC
11610,2016-01-06,20.914528,17002000,20.689640,20.914528,20.464752,ARNC
11611,2016-01-07,21.476746,18307200,20.869549,21.566700,20.847061,ARNC
...,...,...,...,...,...,...,...
12318,2018-12-04,23.480000,2203800,23.129999,23.510000,23.020000,ARNC
12319,2018-12-06,18.000000,3419000,18.139999,18.230000,17.969999,ARNC
12320,2018-12-07,17.299999,2781100,17.250000,17.389999,17.000000,ARNC
12321,2018-12-09,22.690001,2103000,22.270000,22.750000,22.260000,ARNC


In [20]:
sub_split

,Date,Close,Volume,Open,High,Low,Company
0,1970-01-04,6.664637,75300,6.664637,6.711488,6.664637,ARNC
1,1970-01-05,5.997002,83200,6.043853,6.043853,5.973576,ARNC
2,1970-01-06,5.212238,111600,5.059970,5.212238,5.059970,ARNC
3,1970-01-07,4.755435,53300,4.732009,4.802286,4.732009,ARNC
4,1970-01-09,4.802286,132300,4.802286,4.813999,4.778861,ARNC
...,...,...,...,...,...,...,...
11602,2015-12-24,22.758604,11794700,22.848558,23.005980,22.668648,ARNC
11603,2015-12-28,22.443762,15591600,22.488739,22.511227,22.061453,ARNC
11604,2015-12-29,22.781109,5974800,22.736132,23.095951,22.488756,ARNC
11605,2015-12-30,22.398785,23528300,22.353806,22.556206,22.151407,ARNC


In [0]:
# prepare training and testing data sets for LSTM based sequence modeling
#def dimitris_function(time_series, scaling=True,split_ratio=0.9):
def dimitris_function(time_series,split_ratio=0.9):

    series1 = pd.Series(time_series[time_series.columns[0]])
    series2 = pd.Series(time_series[time_series.columns[1]])
    series3 = pd.Series(time_series[time_series.columns[2]])
    series4 = pd.Series(time_series[time_series.columns[3]])
    series5 = pd.Series(time_series[time_series.columns[4]])

    values1 = series1.values
    values1 = values1.reshape((len(values1), 1))
    values2 = series2.values
    values2 = values2.reshape((len(values2), 1))
    values3 = series3.values
    values3 = values3.reshape((len(values3), 1))
    values4 = series4.values
    values4 = values4.reshape((len(values4), 1))
    values5 = series5.values
    values5 = values5.reshape((len(values5), 1))

    #scaler2 --> Close Price Scaler
    scaler1 = MinMaxScaler(feature_range=(0, 1))
    time_series["scaled1"] = scaler1.fit_transform(values1)
    scaler2 = MinMaxScaler(feature_range=(0, 1))
    time_series["scaled2"]= scaler2.fit_transform(values2)
    scaler3 = MinMaxScaler(feature_range=(0, 1))
    time_series["scaled3"]= scaler3.fit_transform(values3)
    scaler4 = MinMaxScaler(feature_range=(0, 1))
    time_series["scaled4"]= scaler4.fit_transform(values4)
    scaler5 = MinMaxScaler(feature_range=(0, 1))
    time_series["scaled5"]= scaler5.fit_transform(values5)


    time_series = time_series[["scaled1","scaled2","scaled3","scaled4","scaled5"]].values

        
    split_ratio = int(len(time_series) * split_ratio)

    train = time_series[0:split_ratio]
    test = time_series[split_ratio:len(time_series)-1]
    
    # return train,test,scaler1,scaler2
    return train,test,scaler2


In [22]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
#def get_seq_model(hidden_units1=40, hidden_units2=65, hidden_units3=32, drop=0.2,input_shape=(1,1)):
def get_seq_model(x_train, y_train, x_val, y_val, params):  

    # create and fit the LSTM network
    model = Sequential()

    # samples*timesteps*features
    # https://keras.io/getting-started/sequential-model-guide/
    #https://journals.plos.org/plosone/article?id=10.1371/journal.pone.0227222

    ######!!!!!#####!!!!!
    #(train_x.shape[1],5) where 5 is the number of features is equal to :
    #(train_x.shape[1],train_x.shape[2])
    model.add(LSTM(
                   input_shape=(x_train.shape[1],x_train.shape[2]),
                   units = params['hidden_1'],  
                   #activation='relu',recurrent_activation='sigmoid',
                   return_sequences=True
    ))
    model.add(Dropout(params['dropout']))
    model.add(LSTM(
                   units = params['hidden_2'],
                   #activation='relu',recurrent_activation='sigmoid',
                   return_sequences=True
    ))
    model.add(Dropout(params['dropout']))   
    model.add(LSTM(
                   units = params['hidden_3'], 
                   #activation='relu',recurrent_activation='sigmoid',
                   return_sequences=True
    ))
    model.add(Dropout(params['dropout']))

    # readout layer. TimeDistributedDense uses the same weights for all
    # time steps.
    # model.add(TimeDistributed(Dense(1))) #number of inputs in the 1st layer
   # model.add(TimeDistributed(Dense(5)))
    model.add(TimeDistributed(Dense(5)))
    start = time.time()
    
    model.compile(loss="mse", optimizer="rmsprop")
    timestr = time.strftime("%Y%m%d-%H%M%S")
    checkpoint_path = "/content/gdrive/My Drive/checkpoints1"  + "/cp-{epoch:04d}.ckpt"
    checkpoint_dir = os.path.dirname(checkpoint_path)
    cp_callback = tf.keras.callbacks.ModelCheckpoint(
        checkpoint_path, verbose=1, save_weights_only=True,#save_best_only='TRUE'
        # Save weights, every 5-epochs.
        period=15)
   # early_stop = keras.callbacks.EarlyStopping(monitor='val_loss', patience=40)
    
    out = model.fit(x_train, y_train,
              epochs=params['epochs'], batch_size=1,
              validation_data=[x_val, y_val],
              #epochs=200, batch_size=1,
              verbose=1,callbacks=[cp_callback])

    return out, model

In [0]:
#Parameters for the model
p = {'hidden_1':[8, 16, 32],
     'hidden_2':[8, 16, 32],
     'hidden_3':[8, 16, 32],
    #  'dropout': [0.1, 0.2],
     'dropout': [0.1],
     'epochs': [100]
     }

In [0]:
# print(type(sub_split.Close))
sub_split_date=sub_split[['Date']]
sub_split=sub_split[['Volume', 'Close', 'Open', 'High', 'Low']]
#sub_split.columns[1]

In [26]:
# split train and test datasets
train,test,scaler_Close = dimitris_function(sub_split,split_ratio=0.7)

#train
print(len(train))
print(train.shape[0])
print(train.shape)
print(test.shape)
print(train)

8124
8124
(8124, 5)
(3482, 5)
[[5.41622228e-04 3.12599479e-02 3.09158294e-02 3.03336467e-02
  3.24798417e-02]
 [6.15015292e-04 2.47806134e-02 2.49575067e-02 2.40463128e-02
  2.56602280e-02]
 [8.78858710e-04 1.71645545e-02 1.55141237e-02 1.62147161e-02
  1.66444716e-02]
 ...
 [1.40543072e-02 7.55340295e-01 7.42492513e-01 7.38870354e-01
  7.60097618e-01]
 [1.51840029e-02 7.79566204e-01 7.87173135e-01 7.75297195e-01
  7.86728811e-01]
 [1.34281435e-02 7.23257359e-01 7.10115205e-01 7.05196676e-01
  7.25477099e-01]]


In [27]:
#We use numpy to reshape our time series into 3D tensors.
train = np.reshape(train,(train.shape[0],1,5))
test = np.reshape(test,(test.shape[0],1,5))

train_x = train[:,:,:]
train_y = train[:,:,:]

test_x = test[:,:,:]
test_y = test[:,:,:]

print("Data Split Complete")

print("train_x shape={}".format(train_x.shape))
print("train_y shape={}".format(train_y.shape))
print("test_x shape={}".format(test_x.shape))
print("test_y shape={}".format(test_y.shape))

Data Split Complete
train_x shape=(8124, 1, 5)
train_y shape=(8124, 1, 5)
test_x shape=(3482, 1, 5)
test_y shape=(3482, 1, 5)


In [0]:
# x=np.concatenate((train_x, test_x), axis=1)
# y=np.concatenate((train_y, test_y), axis=1)
# print("x shape={}".format(x.shape))
# print("y shape={}".format(y.shape))
import talos


In [29]:
#####!!!!!#####!!!!!
#(train_x.shape[1],5) where 5 is the number of features is equal to :
train_x.shape[2]
 #,train_x.shape[2])
#input_shape=(train_x.shape[1],train_x.shape[2])
#input_shape.shape
#validation_data=[(test_x.shape[1],test_x.shape[2]), test_y]
#test=tuple(validation_data)

5

In [0]:
import sklearn

In [0]:
%%capture
# 100	0.0005776109765	0.001970537574	0.1	100	8	16	32
scan_obj = talos.Scan(x=train_x,
                      y=train_y,
                      x_val=test_x,
                      #x_val=train_x,
                      y_val=test_y,
                     #y_val=train_y,

                      params=p,
                      val_split=0.1,
                      model=get_seq_model,
                      #shuffle=False,
                      experiment_name='gdrive/My Drive/test',
                      #fraction_limit=0.1
                      )


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [0]:
print("Finished Talos Scan!!")

In [0]:
#from google.colab import drive
#drive.mount('/content/drive')

In [0]:
scan_obj.data.head()

In [0]:
train_y

In [0]:
# train the model
#https://medium.com/@rajath95/how-to-save-and-upload-deep-learning-machine-learning-models-in-google-colab-using-google-drive-eb2901937ae
# seq_lstm_model.fit(train_x, train_y,
#                 epochs=2, batch_size=1,
#                 #epochs=200, batch_size=1,
#                 verbose=1)
# print("Model Fit Complete")

In [0]:
# train fit performance
trainPredict = seq_lstm_model.predict(train_x)
trainScore = math.sqrt(mean_squared_error(train_y[0], trainPredict[0]))
print('Train Score: %.2f RMSE' % (trainScore))

In [0]:
print()

In [0]:
trainPredict

In [0]:
# Pad input sequence
testPredict = pad_sequences(test_x,maxlen=train_x.shape[1],padding='post',dtype='float64')

In [0]:
# forecast values
testPredict = seq_lstm_model.predict(testPredict)
testPredict

In [0]:
#Drop all other variables except Close Price Output
testPredict=np.delete(testPredict, [2,3,4], 2)
trainPredict=np.delete(trainPredict, [2,3,4], 2)
# testPredict=np.delete(testPredict, 3, 2)
# trainPredict=np.delete(trainPredict, 3, 2)
# testPredict=np.delete(testPredict, 4, 2)
# trainPredict=np.delete(trainPredict, 4, 2)
testPredict=np.delete(testPredict, 0, 2)
trainPredict=np.delete(trainPredict, 0, 2)

In [0]:
testPredict

In [0]:
trainPredict


In [0]:
type(testPredict)

In [0]:
# # inverse transformation of the Close Price
trainPredict = scaler_Close.inverse_transform(trainPredict.\
                                        reshape(-1,1))
testPredict = scaler_Close.inverse_transform(testPredict.\
                                       reshape(-1,1))

In [0]:
trainPredict

In [0]:
# plot the true and forecasted values
split_ratio = len(trainPredict)+1

plt.plot(sub_split_date.Date,
          sub_split.Close.values,c='black',
          alpha=0.3,label='True Data')

plt.plot(sub_split_date.Date[1:split_ratio],
          trainPredict,label='Training Fit',c='g')

plt.plot(sub_split_date.Date[split_ratio+1:],
          testPredict[:test_x.shape[1]],label='Testing Forecast')
plt.title('Forecast Plot')
plt.legend()
#plt.show()
plt.rcParams["figure.figsize"] = (20,10)
plt.show()